In [10]:
import pandas as pd 
from fuzzywuzzy import process
from fuzzywuzzy import fuzz


In [11]:
list_of_vars = ['sponsor', 
                'sponsor_experience', 
                'offering', 
                'offering_city', 
                'offering_state',
                'property_type', 
                'investment_structure', 
                'investment_profile',
                'investor_accreditation',
                '1031_exchange',
                'opportunity_zone',
                'offering_status',
               ]


def lower_and_strip_vars(df, list_of_vars):
    df.columns = df.columns.str.strip().str.lower()\
        .str.replace(' ', '_')\
        .str.replace('?', '')\
        .str.replace('(', '')\
        .str.replace(')', '')\
        .str.replace('%', 'pct')\
        .str.replace('-', '_')\
        .str.replace('$', 'dollars')
    for var in list_of_vars:
        df[var] = df[var].str.lower()
        df[var] = df[var].str.strip()
    return df

def drop_vars(df, list_of_vars):
    df = df.drop(list_of_vars, axis=1)
    return df

    



In [12]:
date_vars = ['date_closed', 
             'targeted_first_distribution_date', 
             'date_funds_due',
             'first_distribution_period_end_date',
             'first_distribution_payment_date', 
             'last_distribution_period_end_date',
             'last_tax_document_date',
             'last_distribution_payment_date',
            ]
def change_to_date_datatypes(df, date_vars):
    for var in date_vars:
        df[var] = pd.to_datetime(df[var])
    return df





In [13]:
def explore_column_w_fuzzwuzzy(df, column):
    choices = list(df[column].unique())
    fuzzy_matches = {}
    for value in choices:
         fuzzy_matches[value] = process.extract(str(value), choices, scorer=fuzz.token_sort_ratio)
    return fuzzy_matches

In [14]:
def check_fuzzy_matches(df, var):
    fuzzy_matches = explore_column_w_fuzzwuzzy(df=df, column=var)
    top_counts = df[var].value_counts().head(20)
    bottom_counts = df[var].value_counts().tail(20)
    return fuzzy_matches, top_counts, bottom_counts

In [15]:
def clean_dirty_column(df, var, old_string, new_string):
    contains_str = df[var].str.contains(old_string)
    df[var] = np.where(contains_str, new_string, df[var])
    return df

In [16]:
def doing_dirty_column_cleaning(df):
    df = clean_dirty_column(df=df,var='offering_city', old_string='multiple', new_string='multiple')
    df = clean_dirty_column(df=df,var='offering_city', old_string='new york', new_string='new york')
    df = clean_dirty_column(df=df,var='offering_city', old_string='atlanta', new_string='atlanta')

    df = clean_dirty_column(df=df,var='offering_state', old_string='multiple', new_string='multiple')
    df = clean_dirty_column(df=df,var='offering_state', old_string='u.s.', new_string='multiple')
    df = clean_dirty_column(df=df,var='offering_state', old_string='ms & al', new_string='multiple')
    df = clean_dirty_column(df=df,var='offering_state', old_string='usa', new_string='multiple')
    df = clean_dirty_column(df=df,var='offering_state', old_string='oregon', new_string='or')
    df = clean_dirty_column(df=df,var='offering_state', old_string='north carolina', new_string='nc')
    df = clean_dirty_column(df=df,var='offering_state', old_string='iowa', new_string='ia')
    df = clean_dirty_column(df=df,var='offering_state', old_string='colorado', new_string='co')

    df = clean_dirty_column(df=df,var='property_type', old_string='mixed use', new_string='multi-use')
    df = clean_dirty_column(df=df,var='property_type', old_string='medical office', new_string='office')
    df = clean_dirty_column(df=df,var='property_type', old_string='flex r&d', new_string='multi-use')

    df = clean_dirty_column(df=df,var='property_type', old_string='residential', new_string='multifamily')
    df = clean_dirty_column(df=df,var='investment_structure', old_string='equity', new_string='equity')
    df = clean_dirty_column(df=df,var='investment_structure', old_string='debt', new_string='debt')
    
    return df

In [17]:
def clean_crowd_street(infile, outfile):
    df = pd.read_csv(infile)
    df = lower_and_strip_vars(df=df,list_of_vars=list_of_vars)
    df = drop_vars(df=df, list_of_vars=['disposition_date', 'realized_investor_irr_pct'])
    df = change_to_date_datatypes(df=df, date_vars=date_vars)
    df = doing_dirty_column_cleaning(df=df)
    
    #cleaning discrepency in reporting percentages
    df['realized_avg_annual_cash_yield_pct'] = np.where(df['realized_avg_annual_cash_yield_pct'] > 0,\
         df['realized_avg_annual_cash_yield_pct'] / 100,\
         df['realized_avg_annual_cash_yield_pct'])
    
    df.to_csv(outfile, index=False)

    return df

In [18]:
clean_crowd_street(infile="../data/raw/Offerings_2019-10-25.csv", 
                   outfile="../data/processed/clean_crowdstreet.csv"
                  )

,sponsor,sponsor_experience,offering,offering_city,offering_state,property_type,investment_structure,investment_profile,targeted_investor_irr_pct,targeted_investor_equity_multiple,...,date_funds_due,date_closed,offering_status,first_distribution_period_end_date,first_distribution_payment_date,last_distribution_period_end_date,last_distribution_payment_date,last_tax_document_date,realized_investor_equity_multiple,realized_avg_annual_cash_yield_pct
0,crowdstreet investments,blended,crowdstreet blended portfolio - series v,multiple,or,multi-use,equity,value add,18.00,2.00,...,NaT,NaT,funding,NaT,NaT,NaT,NaT,NaT,NaN,NaN
1,crowdstreet advisors,blended,private managed accounts by crowdstreet advisors,portland,or,multi-use,equity,value add,NaN,NaN,...,NaT,NaT,funding,NaT,NaT,NaT,NaT,NaT,NaN,NaN
2,crowdstreet advisors,blended,crowdstreet opportunity zone portfolio,multiple,multiple,multi-use,equity,opportunistic,10.00,2.00,...,2019-11-07,2019-11-04,funding,NaT,NaT,NaT,NaT,NaT,NaN,NaN
3,trion properties,seasoned,meadows apartments,newark,ca,multifamily,equity,value add,15.90,1.50,...,2019-11-05,2019-11-01,funding,NaT,NaT,NaT,NaT,NaT,NaN,NaN
4,capstone realty group,tenured,1700 palm beach lakes,west palm beach,fl,office,equity,value add,19.10,2.00,...,2019-10-30,NaT,funding,NaT,NaT,NaT,NaT,NaT,NaN,NaN
5,westpac,tenured,elevation at okemos (ph ii),okemos,mi,multifamily,equity,development,18.00,1.60,...,2019-11-29,2019-11-26,funding,NaT,NaT,NaT,NaT,NaT,NaN,NaN
6,cascadia senior living & development,seasoned,fieldstone memory care - puyallup,puyallup,wa,senior housing,equity,development,22.50,1.80,...,2019-11-29,2019-11-26,funding,NaT,NaT,NaT,NaT,NaT,NaN,NaN
7,one real estate investment,seasoned,axiom apartments charlotte,charlotte,nc,multifamily,equity,core plus,14.70,1.90,...,2019-11-08,2019-11-05,funding,NaT,NaT,NaT,NaT,NaT,NaN,NaN
8,ssi,tenured,61 state street,charleston,sc,hospitality,equity,value add,17.40,1.90,...,2019-11-15,2019-11-12,funding,NaT,NaT,NaT,NaT,NaT,NaN,NaN
9,middleburg real estate partners,tenured,mosby riverlights,wilmington,nc,multifamily,equity,development,20.40,1.70,...,2019-11-08,2019-11-05,funding,NaT,NaT,NaT,NaT,NaT,NaN,NaN
